# TODOs
- Add hidden flag to Parts to control whether a Part will be shown initially
- Add color box to parts in navigator
- Fix toggle isssue when mixing toggle and wire
- OrthoViewPoint
- Extend AxisView with id

# Library

In [1]:
import cadquery as cq
from cq_jupyter import Assembly, Part, DEBUG

import cq_jupyter
cq_jupyter.cq_jupyter.DEBUG = False

In [2]:
b = cq.Workplane('XY')
box1 = b.box(1,2,3).edges(">X").chamfer(0.1)
box2 = b.transformed(offset=cq.Vector(0, 2, 1)).box(3,2,1).edges(">Z").fillet(0.1)

Assembly(
    Part(box1, "redbox", (1,0,0)), 
    Part(box2, "greenbox", (0,1,0)),
    height=600
)

View id: 36099e5f-c657-447a-965d-58e6aa5cde54


In [ ]:
from OCC.gp import gp_Vec, gp_Quaternion

def _gp_Vec_repr(self):
    return "gp_Vec(x=%f, y=%f, z=%f)" % (self.X(), self.Y(), self.Z())

def _gp_Quaternion_repr(self):
    return "gp_Quaternion(x=%f, y=%f, z=%f, w=%f)" % (self.X(), self.Y(), self.Z(), self.W())

gp_Vec.__repr__ = _gp_Vec_repr

gp_Quaternion.__repr__ = _gp_Quaternion_repr

gp_Vec(0,1,2), gp_Quaternion(0,1,2,3)


In [ ]:
from OCC.Display.WebGl.x3dom_renderer import X3DExporter
from OCC.gp import gp_Quaternion, gp_Vec
from uuid import uuid4
from math import tan, atan, sqrt, pi
import json

from cadquery.occ_impl.geom import BoundBox

N_HEADER_LINES = 11
CSS = """
.Row {
    display: table;
    width: 100%;
    table-layout: fixed;
    border-spacing: 10px;
}
.ViewerColumn {
    display: table-cell;
    width: 85%;
    vertical-align: top;
}
.NavColumn {
    display: table-cell;
    width: 15%;
    vertical-align: top;
    overflow: scroll;
}
#x3domAxisSceneView {
    position: absolute;
    width: 80px;
    height: 80px;
    left: 320px;
    bottom: 60px;
    border: none;
    z-index: 1000;
}
"""

JS="""
function ViewConnector (scene, options)
{
    var that = this;

    /**
     * Initialize the ViewConnector
     */

    this.active = null;

    this.init = function ()
    {
        //Set default options
        this._options = $.extend( {}, $.fn.viewConnector.defaults, options );

        //Check if we have a connected scene
        if (this._options.connected == null)
        {
            //Throw error
            console.error("[ViewConnector] No connected scene available!");
        }
        else
        {
            //Set scene
            this._scene = scene;

            //Set connected scene
            this._connectedScene = document.getElementById(this._options.connected);

            //Set viewpoint
            this._viewMatrix = this._scene.runtime.viewpoint()._viewMatrix;

            //Add viewpointChanged listeners
            this.addViewpointChangedListeners();
        }
    };

    /**
     * Add viewpointChanged listeners to all viewpoints that are bind to the connected scene
     */
    this.addViewpointChangedListeners = function()
    {   
        /*
        var bindBag = this._connectedScene.runtime.viewpoint()._stack._bindBag;
        for (var i=0; i<bindBag.length; i++)
        {
            bindBag[i]._xmlNode.addEventListener("viewpointChanged", that.viewPointChangedHandler);
        }
        */
        var self = this;
        var origin = this._scene.runtime,
            dest = this._connectedScene.runtime;

        /* TODO add orthoviewpoint to the selector */
//        $(this._scene).find('orthoviewpoint').on('viewpointChanged', function(event) {
        $(this._scene).find('viewpoint').on('viewpointChanged', function(event) {
            if(self.active != origin && self.active != null ) return;
            self.active = origin;
            clearTimeout(self._reset);
            self._reset = setTimeout(self.resetActive.bind(self), 60);


            self.setOrientation(origin, dest, event.originalEvent)
        });

        /* TODO add orthoviewpoint to the selector */
//        $(this._connectedScene).find('orthoviewpoint').on('viewpointChanged', function(event) {
        $(this._connectedScene).find('viewpoint').on('viewpointChanged', function(event) {
            if(self.active != dest && self.active != null ) return;
            self.active = dest;
            clearTimeout(self._reset);
            self._reset = setTimeout(self.resetActive.bind(self), 60);
            self.setOrientation(dest, origin, event.originalEvent)
        })
    };

    this._reset = -1;
    this.resetActive = function() {
        this.active = null;
    }


    this.setOrientation = function(origin, dest, event) {
         try {

            var SFMatrix4f = x3dom.fields.SFMatrix4f;
            var SFVec3f = x3dom.fields.SFVec3f;

            var viewpoint = dest.viewpoint(),
                originVp = origin.viewpoint(),
                originVm = origin.viewMatrix();

            var _vp = dest.viewpoint();
            var viewpointPosition = dest.viewMatrix().inverse().e3(),
                distanceToCoR = _vp.getCenterOfRotation().subtract( viewpointPosition ).length();

            // Taken from x3dom fire viewpointChanged
            var e_viewtrafo = originVp.getCurrentTransform();
            e_viewtrafo = e_viewtrafo.inverse().mult(originVm);
            var e_mat = e_viewtrafo.inverse();
            var e_rotation = new x3dom.fields.Quaternion(0, 0, 1, 0);
            e_rotation.setValue(e_mat);

            var upVector = e_rotation.toMatrix().e1();
            var destPos =   viewpoint.getCenterOfRotation().add(
                                originVm.inverse().e3().subtract( 
                                    originVp.getCenterOfRotation()
                                ).normalize().multiply(distanceToCoR)
                            );

            var pos = SFMatrix4f.lookAt( destPos, viewpoint.getCenterOfRotation(), upVector );
            dest.canvas.doc._viewarea._transMat =  SFMatrix4f.identity();
            dest.canvas.doc._viewarea._rotMat =  SFMatrix4f.identity();
            dest.canvas.doc._viewarea._movement = new SFVec3f(0, 0, 0);

            _vp.setView( pos.inverse() )

            dest.triggerRedraw();
        // errors are not caught somewhere else and it's difficult to debug
        } catch(e) {
            console.error(e)
        }
    }

    /**
     * Handles viewpointChanged events from all viewpoints that are bind to the connected scene
     * @param event
     */
    this.viewPointChangedHandler = function(event)
    {
        //Update position
        if (that._options.connectPosition)
        {
            that._viewMatrix._03 = event.matrix._03;
            that._viewMatrix._13 = event.matrix._13;
            that._viewMatrix._23 = event.matrix._23;
        }

        //Update orientation
        if (that._options.connectOrientation)
        {
            that._viewMatrix._00 = event.matrix._00;
            that._viewMatrix._01 = event.matrix._01;
            that._viewMatrix._02 = event.matrix._02;

            that._viewMatrix._10 = event.matrix._10;
            that._viewMatrix._11 = event.matrix._11;
            that._viewMatrix._12 = event.matrix._12;

            that._viewMatrix._20 = event.matrix._20;
            that._viewMatrix._21 = event.matrix._21;
            that._viewMatrix._22 = event.matrix._22;

            that._viewMatrix._30 = event.matrix._30;
            that._viewMatrix._31 = event.matrix._31;
            that._viewMatrix._32 = event.matrix._32;
        }

        //Trigger redraw
        that._scene.runtime.triggerRedraw();
    };

    //Finally initialize the editor
    this.init();
}

$.fn.viewConnector = function(options) {
    var args = arguments;

    if (options === undefined || typeof options === 'object') {
        // Creates a new plugin instance, for each selected element, and
        // stores a reference within the element's data
        return this.each(function() {
            if (!$.data(this, 'ViewConnector')) {
                $.data(this, 'ViewConnector', new ViewConnector(this, options));
            }
        });
    } else if (typeof options === 'string' && options[0] !== '_' && options !== 'init') {
        // Call a public plugin method (not starting with an underscore) for each
        // selected element.
        if (Array.prototype.slice.call(args, 1).length == 0 && $.inArray(options, $.fn.editor2D.getters) != -1) {
            // If the user does not pass any arguments and the method allows to
            // work as a getter then break the chainability so we can return a value
            // instead the element reference.
            var instance = $.data(this[0], 'ViewConnector');
            return instance[options].apply(instance, Array.prototype.slice.call(args, 1));
        } else {
            // Invoke the specified method on each selected element
            return this.each(function() {
                var instance = $.data(this, 'ViewConnector');
                if (instance instanceof ViewConnector && typeof instance[options] === 'function') {
                    instance[options].apply(instance, Array.prototype.slice.call(args, 1));
                }
            });
        }
    }
    return null;
};

// Plugin defaults – added as a property on our plugin function.
$.fn.viewConnector.defaults = {
    connected: null,
    connectPosition: true,
    connectOrientation: true
};
    
function findShape(id, name) {
    cq = $('#cq_x3d_column_' + id);
    children = cq.find("scene").children();
    shape = null;
    swtch = null;
    for (i=0; i<children.length; i++) {
        if (children[i].getAttribute("DEF") == name) {
            shape = children[i];
            swtch = children[i+1];
            break;
        }
    }
    return [shape, swtch]
}

function show(dir) {
    var id = '%s';
    e = document.getElementById(id)
    e.runtime.showAll(dir)
}

function refit() {
    e = document.getElementById('%s')
    e.runtime.fitAll(false)
}

function reset() {
    e = document.getElementById('%s')
    e.runtime.resetView()
}

function setup_nav(id, parts) {
    nav = $('#cq_nav_column_' + id);
    html = ""
    for (part in parts) {
        html += "<div>"
        html += "<span onclick=\\"toggle('" + parts[part] + "')\\">[toggle] </span>"
        html += "<span onclick=\\"wire('" + parts[part] + "')\\"> [wire] </span>"
        html += "<span> " + part + " </span>"
        html += "</div>"
    };
    nav.html(html);
    console.log(id);
    
    $("#x3domAxisSceneView").viewConnector({connected: id, connectPosition: false});
}

function toggle(name) {
    var id = '%s'
    shape = findShape(id, name)
    if (shape[0].getAttribute("render") == "true") {
        shape[0].setAttribute("render", false)
    } else {
        shape[0].setAttribute("render", true)
    }

    if (shape[1].getAttribute("whichChoice") == 0) {
        shape[1].setAttribute("whichChoice", -1)
    } else {
        shape[1].setAttribute("whichChoice", 0)
    }
}

function wire(name) {
    var id = '%s'
    shape = findShape(id, name)
    if (shape[0].getAttribute("render") == "true") {
        shape[0].setAttribute("render", false)
    } else {
        shape[0].setAttribute("render", true)
    }

    shape[1].setAttribute("whichChoice", 0)
}

if (document.getElementById('X3DOM_JS_MODULE') == null){
    var scr  = document.createElement('script');
    head = document.head || document.getElementsByTagName('head')[0];
    scr.src = 'http://www.x3dom.org/download/x3dom.js';
    scr.async = false;
    scr.id = 'X3DOM_JS_MODULE';
    scr.onload = function () {
        x3dom.reload();
    }
    head.insertBefore(scr, head.lastChild);      
}
else if (typeof x3dom != 'undefined') { //call reload only if x3dom already loaded
    x3dom.reload();
}
setup_nav('%s', %s);
"""

BOILERPLATE = \
'''
<link rel='stylesheet' type='text/css' href='http://www.x3dom.org/download/x3dom.css'></link>
<style>
    {css}
</style>
<script>
    {js}
</script>
<div style='height: {divheight}px; width: 100%;' width='100%' height='{divheight}px' id='cq_viewer_{id}'>
    <div class="Row">
        <span onclick="refit()"> [fit]</span>
        <span onclick="reset()"> [reset]</span>
        <span onclick="show('posX')"> [right]</span>
        <span onclick="show('negX')"> [left]</span>
        <span onclick="show('posY')"> [front]</span>
        <span onclick="show('negY')"> [back]</span>
        <span onclick="show('posZ')"> [bottom]</span>
        <span onclick="show('negZ')"> [top]</span>
    <div>
    <div class="Row" id='cq_viewer_row_{id}'>
        <div class="NavColumn" id='cq_nav_column_{id}'>
            abc
        </div>
        <div class="ViewerColumn" id='cq_x3d_column_{id}'>
            <x3d class="cqviewer" style='height: {height}px; width: 100%;' id='{id}' width='100%'>
                <scene>
                    <Viewpoint  position='{x},{y},{z}' centerOfRotation='{x0} {y0} {z0}' orientation='{rot}' fieldOfView='{fov}'></Viewpoint>
                    <OrthoViewpoint  position='{x},{y},{z}' centerOfRotation='{x0} {y0} {z0}' orientation='{rot}' fieldOfView='{fov}'></OrthoViewpoint>
                    {src}
                </scene>
            </x3d>        
        </div>
        <div>
            <x3d id="x3domAxisSceneView">
                <scene>
                    <navigationInfo type='"NONE" "ANY"'></navigationInfo>
                    <viewpoint position="0 0 5.0"  orientation="0 0 0 1"></viewpoint>
                    <Transform id="axes">        
                        <!-- X arrow and label -->
                        <Shape isPickable="false" DEF="AXIS_LINE_X">
                            <IndexedLineSet index="0 1 -1">
                                <Coordinate point="0 0 0, 1 0 0" color="1 0 0, 1 0 0"></Coordinate>
                            </IndexedLineSet>
                            <Appearance DEF='Red'><Material diffuseColor="0 0 0" emissiveColor='1 0 0'></Material></Appearance>
                        </Shape>
                        <Transform translation='1 0 0'>
                            <Transform rotation='1 0 0 1.57079632679'>
                                <Billboard>
                                    <Shape isPickable="false" DEF="AXIS_LABEL_X">
                                        <Text string="X" solid="false">
                                            <FontStyle size="0.6"></FontStyle>
                                        </Text>
                                        <Appearance USE='Red'></Appearance>
                                    </Shape>
                                </Billboard>
                            </Transform>
                        </Transform>


                        <!-- Y arrow and label -->                            
                        <Shape isPickable="false" DEF="AXIS_LINE_Y">
                            <IndexedLineSet index="0 1 -1">
                                <Coordinate point="0 0 0., 0 1 0" color="0 1 0, 0 1 0"></Coordinate>
                            </IndexedLineSet>
                            <Appearance DEF='Green'><Material diffuseColor="0 0 0" emissiveColor='0 1 0'></Material></Appearance>
                        </Shape>
                        <Transform translation='0 1 0'>
                            <Transform rotation='1 0 0 1.57079632679'>
                                <Billboard>
                                    <Shape isPickable="false" DEF="AXIS_LABEL_Y">
                                        <Text string="Y" solid="false">
                                            <FontStyle size="0.6"></FontStyle>
                                        </Text>
                                        <Appearance USE='Green'></Appearance>
                                    </Shape>
                                </Billboard>
                            </Transform>
                        </Transform>


                        <!-- Z arrow and label -->
                        <Shape isPickable="true" DEF="AXIS_LINE_Z">
                            <IndexedLineSet index="0 1 -1">
                                <Coordinate point="0 0 0, 0 0 1" color="0 0 1, 0 0 1"></Coordinate>
                            </IndexedLineSet>
                            <Appearance DEF='Blue'><Material diffuseColor="0 0 0" emissiveColor='0 0 1'></Material></Appearance>
                        </Shape>
                        <Transform translation='0 0 1'>
                            <Transform rotation='1 0 0 1.57079632679'>
                                <Billboard>
                                    <Shape isPickable="false" DEF="AXIS_LABEL_Z">
                                        <Text string="Z" solid="false">
                                            <FontStyle size="0.6"></FontStyle>
                                        </Text>
                                        <Appearance USE='Blue'></Appearance>
                                    </Shape>
                                </Billboard>
                            </Transform>
                        </Transform>
                    </Transform>
                </scene>
            </x3d> 
        <div>
    </div>
</div>
'''

#https://stackoverflow.com/questions/950087/how-do-i-include-a-javascript-file-in-another-javascript-file
#better if else

FOV = 0.1

# See https://en.wikipedia.org/wiki/Isometric_projection
def axon():
    alpha = atan(1/sqrt(2))
    q1 = gp_Quaternion(gp_Vec(1,0,0), alpha)
    q2 = gp_Quaternion(gp_Vec(0,0,1), pi/4)
    q = q2*q1
    axis = gp_Vec(0.0, 0.0, 0.0)
    angle = gp_Quaternion.GetVectorAndAngle(q, axis)
    
    return (axis, angle)

def add_x3d_boilerplate(src, parts, height=400, center=(0,0,0), d=(0,0,15), fov=FOV, rot='0 0 1 0 '):
    print("\ncenter:", center, "\nd:", d, "\nrot:", rot)
    id = uuid4()
    print("id:", id)
    js = JS % (id, id, id, id, id, id, json.dumps({part: "shape%d"%i for (i, part) in enumerate(parts)}))
    html = BOILERPLATE.format(css=CSS,
                              js=js,
                              src=src,
                              parts=parts,
                              id=id,
                              height=height,
                              divheight=height+100,
                              x=d[0],
                              y=d[1],
                              z=d[2],
                              x0=center[0],
                              y0=center[1],
                              z0=center[2],
                              fov=fov,
                              rot=rot)
    # print(html)
    return html

def x3d_display(*parts,
                vertex_shader=None,
                fragment_shader=None,
                export_edges=True,
                specular_color=(1,1,1),
                shininess=0.4,
                transparency=0.4,
                line_color=(0,0,0),
                line_width=2.,
                mesh_quality=.3,
                height=400):

        x3d_str = ""
        for i, part in enumerate(parts):
            exporter = X3DExporter(part.shape.wrapped,
                                   vertex_shader,
                                   fragment_shader,
                                   export_edges,
                                   part.color,
                                   part.color,
                                   shininess,
                                   transparency,
                                   line_color,
                                   line_width,
                                   mesh_quality)

            exporter.compute()
            x3d_str_shape = exporter.to_x3dfile_string(i)
            x3d_str += '\n'.join(x3d_str_shape.splitlines()[N_HEADER_LINES:-2]) +"\n"

        compound = Compound.makeCompound([part.shape for part in parts]).wrapped
        
        bb = BoundBox._fromTopoDS(compound)
        d = [0.5*b for b in (bb.xlen, bb.ylen, bb.zlen)]
        c = bb.center
    
        axis, angle = axon() # gp_Vec(0,0,1), 0 
        q = gp_Quaternion(axis, angle)
        viewDir = q.Multiply(gp_Vec(0,0,1))
        tanfov2 = tan(FOV / 2.0)
        dist = sqrt(sum([x*x for x in d])) / tanfov2
        vec  = c.wrapped + (viewDir * dist)

        return add_x3d_boilerplate(x3d_str,
                                   [part.name for part in parts],
                                   d=(vec.X(),vec.Y(),vec.Z()),
                                   center=(c.x,c.y,c.z),
                                   height=height,
                                   rot='{} {} {} {} '.format(axis.X(), axis.Y(), axis.Z(), angle)
                                  )
    

In [ ]:
from cadquery import Shape, Compound

def _repr_html_(self):
    """
    Jupyter 3D representation support
    """
    return x3d_display(Part(self, "shape0", (1, 1, 0)), export_edges=True)

Shape._repr_html_ = _repr_html_

class Part(object):
    def __init__(self, shape, name, color=(1,1,0)):
        self.shape = shape
        self.name = name
        self.color = color
        
class Assembly(object):
    def __init__(self, *parts, height=400):
        self.parts = parts
        self.height = height
        
    def _repr_html_(self):
        assembly = []
        for part in self.parts:
            # Replace original shape with compound
            assembly.append(Part(Compound.makeCompound(part.shape.objects), part.name, part.color))

        return x3d_display(*assembly, export_edges=True, height=self.height)

# Test

In [ ]:
import cadquery as cq

In [ ]:
b = cq.Workplane('XY')
box1 = b.box(1,2,3).edges(">X").chamfer(0.1)
box2 = b.transformed(offset=cq.Vector(0, 2, 1)).box(3,2,1).edges(">Z").fillet(0.1)

Assembly(
    Part(box1, "redbox", (1,0,0)), 
    Part(box2, "greenbox", (0,1,0)),
    height=600
)


# iPhone Halterung

In [ ]:
length = 70
width = 80
height = 8.5
thickness = 1.5
deviceRadius = 7
innerRadius = (height - 0.5) / 2.0
padding = 11.0 - deviceRadius

b = cq.Workplane('XY')
f = cq.Workplane('XZ')

inner = (
    b.box(length, width, height)
       .edges('|Z').edges('<Y').fillet(deviceRadius)
       .edges("|Y").edges("<Y").fillet(innerRadius)
)

outer = (
    b.box(length + thickness, width + 2 * thickness, height + thickness)
        .edges('|Z').edges('<Y').fillet(deviceRadius + thickness / 2)  
        .edges("|Y").edges("<Y").fillet(innerRadius + thickness/2)
)

front_cutter = (
    f.transformed(offset=cq.Vector(0, 0, -width/2))
    .box(length + 2 * thickness, height + 2 * thickness, 2 * thickness)
)

top_cutter = (
    b.transformed(offset=cq.Vector(0, thickness/2, (height)/2))
    .box(length - deviceRadius, width - deviceRadius + 2* thickness, 2 * thickness)
    .edges('|Z').edges('<Y').fillet(innerRadius)  
)

bottom_cutter = (
    b.transformed(offset=cq.Vector(0, -width/2 + thickness, thickness))
    .box(length - deviceRadius - padding, deviceRadius + 2 * thickness, height + thickness)
    .edges('|Y').edges('<Y').fillet(height/2 + thickness/4)
)

shell = (
    outer
    .cut(inner)
    .cut(front_cutter)
    .cut(top_cutter)
    .cut(bottom_cutter)
)

Assembly(
    Part(shell, "shell", (1,1,0)), 
    Part(front_cutter, "front_cutter", (1,0,0)),
    Part(top_cutter, "top_cutter", (0,1,0)),
    Part(bottom_cutter, "bottom_cutter", (0,0,1)),
    height=800
)

In [ ]:
from cadquery import exporters

In [ ]:
with open("hh.stl", "w") as fd:
    exporters.exportShape(shell, "STL", fd)

# Experiments
## Coord-System

In [ ]:
from IPython.display import HTML

CSS = """
#x3domAxisSceneView {
    position: absolute;
    width: 100px;
    height: 100px;
    left: 100px;
    bottom: 0px;
    border: none;
    z-index: 1000;
}
"""

JS = """
if (document.getElementById('X3DOM_JS_MODULE') == null){
    var scr  = document.createElement('script');
    head = document.head || document.getElementsByTagName('head')[0];
    scr.src = 'http://www.x3dom.org/download/x3dom.js';
    scr.async = false;
    scr.id = 'X3DOM_JS_MODULE';
    scr.onload = function () {
        x3dom.reload();
    }
    head.insertBefore(scr, head.lastChild);      
}
else if (typeof x3dom != 'undefined') { //call reload only if x3dom already loaded
    x3dom.reload();
}
"""

html = """
<link rel='stylesheet' type='text/css' href='http://www.x3dom.org/download/x3dom.css'></link>
<style>
%s
</style>
<script>
%s
</script>
<div style='height: 100px; width: 100%%;' width='100%%' height='100px' id='coord'>


    <x3d id="x3domAxisSceneView">
        <scene>
            <navigationInfo type='"NONE" "ANY"'></navigationInfo>
            <viewpoint position="0 0 2.0"  orientation="0 0 0 1"></viewpoint>
            <Transform id="axes">        
                <!-- X arrow and label -->
                <Shape isPickable="false" DEF="AXIS_LINE_X">
                    <IndexedLineSet index="0 1 -1">
                        <Coordinate point="0 0 0, 1 0 0" color="1 0 0, 1 0 0"></Coordinate>
                    </IndexedLineSet>
                    <Appearance DEF='Red'><Material diffuseColor="0 0 0" emissiveColor='1 0 0'></Material></Appearance>
                </Shape>
                <Transform translation='1 0 0'>
                    <Transform rotation='1 0 0 1.57079632679'>
                        <Billboard>
                            <Shape isPickable="false" DEF="AXIS_LABEL_X">
                                <Text string="X" solid="false">
                                    <FontStyle size="0.6"></FontStyle>
                                </Text>
                                <Appearance USE='Red'></Appearance>
                            </Shape>
                        </Billboard>
                    </Transform>
                </Transform>


                <!-- Y arrow and label -->                            
                <Shape isPickable="false" DEF="AXIS_LINE_Y">
                    <IndexedLineSet index="0 1 -1">
                        <Coordinate point="0 0 0., 0 1 0" color="0 1 0, 0 1 0"></Coordinate>
                    </IndexedLineSet>
                    <Appearance DEF='Green'><Material diffuseColor="0 0 0" emissiveColor='0 1 0'></Material></Appearance>
                </Shape>
                <Transform translation='0 1 0'>
                    <Transform rotation='1 0 0 1.57079632679'>
                        <Billboard>
                            <Shape isPickable="false" DEF="AXIS_LABEL_Y">
                                <Text string="Y" solid="false">
                                    <FontStyle size="0.6"></FontStyle>
                                </Text>
                                <Appearance USE='Green'></Appearance>
                            </Shape>
                        </Billboard>
                    </Transform>
                </Transform>


                <!-- Z arrow and label -->
                <Shape isPickable="true" DEF="AXIS_LINE_Z">
                    <IndexedLineSet index="0 1 -1">
                        <Coordinate point="0 0 0, 0 0 1" color="0 0 1, 0 0 1"></Coordinate>
                    </IndexedLineSet>
                    <Appearance DEF='Blue'><Material diffuseColor="0 0 0" emissiveColor='0 0 1'></Material></Appearance>
                </Shape>
                <Transform translation='0 0 1'>
                    <Transform rotation='1 0 0 1.57079632679'>
                        <Billboard>
                            <Shape isPickable="false" DEF="AXIS_LABEL_Z">
                                <Text string="Z" solid="false">
                                    <FontStyle size="0.6"></FontStyle>
                                </Text>
                                <Appearance USE='Blue'></Appearance>
                            </Shape>
                        </Billboard>
                    </Transform>
                </Transform>
            </Transform>
        </scene>
    </x3d> 
""" % (CSS, JS)
HTML(html)

## Quaternions

In [ ]:
from OCC.gp import gp_Quaternion, gp_Vec
from pyquaternion import Quaternion
import pyquaternion
import numpy as np
from math import pi

def tostr(q):
    return "%9.6f %9.6f %9.6f %9.6f " % (q.W(), q.X(), q.Y(), q.Z())

def todict(q):
    if isinstance(q, Quaternion):
        return dict(x=q[1], y=q[2], z=q[3], w=q[0])
    else:
        return dict(x=q.X(), y=q.Y(), z=q.Z(), w=q.W())

In [ ]:
front = gp_Quaternion(gp_Vec(0,0,1), 0)
left = gp_Quaternion(gp_Vec(0,0,1), pi/2)
back = gp_Quaternion(gp_Vec(0,0,1), pi)
right = gp_Quaternion(gp_Vec(0,0,1), -pi/2)
top = gp_Quaternion(gp_Vec(1,0,0), pi/2)
bottom = gp_Quaternion(gp_Vec(1,0,0), -pi/2)

print(tovec(front))
print(tovec(left))
print(tovec(back))
print(tovec(right))
print(tovec(top))
print(tovec(bottom))

q1 = gp_Quaternion(gp_Vec(1,0,0), -pi/4)
q2 = gp_Quaternion(gp_Vec(0,1,0), -pi/4)
axio = q1.Multiplied(q2)
print(tovec(axio))

In [ ]:
Quaternion(axis=[1,1,1], angle=0).transformation_matrix

In [ ]:
q1 = Quaternion(axis=[1, 0, 0], angle=-pi/4)
q2 = Quaternion(axis=[0, 1, 0], angle=-pi/4)
np.dot(q1.transformation_matrix, q1.elements)


In [ ]:
q = Quaternion(0.77, 0.3, 0.55, 1.28).normalised
q, q.angle, q.axis


In [ ]:
def tovec(q):
    return (q.W(), q.X(), q.Y(), q.Z())

FRONT = tovec(gp_Quaternion(gp_Vec(0,0,1), 0))
LEFT = tovec(gp_Quaternion(gp_Vec(0,0,1), pi/2))
BACK = tovec(gp_Quaternion(gp_Vec(0,0,1), pi))
RIGHT = tovec(gp_Quaternion(gp_Vec(0,0,1), -pi/2))
TOP = tovec(gp_Quaternion(gp_Vec(1,0,0), pi/2))
BOTTOM = tovec(gp_Quaternion(gp_Vec(1,0,0), -pi/2))
AXIO = tovec(gp_Quaternion(gp_Vec(1,0,0), -pi/6).Multiplied(
       gp_Quaternion(gp_Vec(0,1,0), -pi/6)))

In [ ]:
class Vec():
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

def translation(vec):
    return np.array((
        [1, 0, 0, vec.x],
        [0, 1, 0, vec.y],
        [0, 0, 1, vec.z],
        [0, 0, 0, 1]
    ))

def toMatrix(x, y, z, w):
    xx = x * x
    xy = x * y
    xz = x * z
    yy = y * y
    yz = y * z
    zz = z * z
    wx = w * x
    wy = w * y
    wz = w * z

    return np.array((
        [1 - 2 * (yy + zz),     2 * (xy - wz),     2 * (xz + wy), 0],
        [    2 * (xy + wz), 1 - 2 * (xx + zz),     2 * (yz - wx), 0],
        [    2 * (xz - wy),     2 * (yz + wx), 1 - 2 * (xx + yy), 0],
        [                0,                 0,                 0, 1]
    ))

m1 = translation(Vec(16.44089884623471, -17.207618072099322, 13.306594890751247))
m2 = toMatrix(0.4632345357781693, 0.1804809879655205, 0.33088181127012095, 0.8020957578842927)
vm = np.linalg.inv(m1*m2)
vm

In [ ]:
q = Quaternion(0.4632345357781693, 0.1804809879655205, 0.33088181127012095, 0.8020957578842927)
q.transformation_matrix

In [ ]:
q._q_matrix()

In [ ]:
q._q_bar_matrix()

In [ ]:
np.linalg.inv(vm)

In [ ]:
position = (16.44089884623471, -17.207618072099322, 13.306594890751247)
centerOfRotation = (0.0, -0.4999999999999999, 0.4999999999999999)
orientation = (0.77, 0.3, 0.55, 1.28)
fieldOfView = 0.2

In [ ]:
q = Quaternion(*orientation)

In [ ]:
q.normalised

In [ ]:
import math

def rotateFromTo(f, t):
    cost = f.dot(to_)
    axis = np.cross(from_, t)
    axis = axis / np.linalg.norm(axis)

    s = math.sqrt(0.5 * (1.0 - cost));
    axis = s*axis
    s = math.sqrt(0.5 * (1.0 + cost));
    return (*axis, s)

from_ = np.array([0,0,1])
to_ = np.array([1,1,0])
to_ = to_ / np.linalg.norm(to_)
rotateFromTo(from_, to_)

In [ ]:
fov = 0.2
tanfov2 = math.tan(fov / 2.0)
center = (0, -0.5, 0.5)

In [ ]:
q = Quaternion(0.4247081321999479, 0.1759200437218226, 0.339851090706265, 0.8204732639190053)

In [ ]:
q.axis

In [ ]:
import math 

alpha = math.atan(1/math.sqrt(2))
r1 = Quaternion(axis=(1,0,0), angle=alpha)
r2 = Quaternion(axis=(0,0,1), angle=pi/4)
r = r2 * r1
todict(r1), todict(r2), r1.transformation_matrix , r2.transformation_matrix, todict(r).values(), r.angle, r.axis

In [ ]:
q1 = gp_Quaternion(gp_Vec(1,0,0), alpha)
q2 = gp_Quaternion(gp_Vec(0,0,1), pi/4)
q = q2*q1
axis = gp_Vec(0.0, 0.0, 0.0)
angle = gp_Quaternion.GetVectorAndAngle(q, axis)
todict(q1), todict(q2), todict(q).values(), angle, axis.X(), axis.Y(), axis.Z()


In [ ]:
r1.q

In [ ]:
p = r1*r2

In [ ]:
r.axis

In [ ]:
r.rotation_matrix * (0,0,1)

In [ ]:
q1 = gp_Quaternion(gp_Vec(0,1,0), pi/4)
q2 = gp_Quaternion(gp_Vec(1,0,0), math.atan(1/math.sqrt(2)))
axio = q1.Multiplied(q2)
tovec(axio)

In [ ]:
math.sin(alpha)

In [ ]:
1/math.sqrt(2), 1/math.sqrt(3), 1/math.sqrt(6)

In [ ]:
tan(0.1)

In [ ]:
# FreeCAD
Front = (-0.7071067811865475, 0.0, 0.0, -0.7071067811865475)
Top = (0.0, 0.0, 0.0, 1.0)
Right = (0.5, 0.5, 0.5, 0.5)
Rear = (0.0, 0.7071067811865475, 0.7071067811865475, 0.0)
Bottom = (0.0, -1.0, 0.0, 0.0)
Left = (-0.5, 0.5, 0.5, -0.5)

In [ ]:
for v in (Front, Top, Right, Rear, Bottom, Left):
    q = Quaternion(*v)
    print(q.angle, q.axis)

In [ ]:
np.array([0.77, 0.3, 0.55, 1.28]) / np.array([0.46323453577816937, 0.1804809879655206, 0.330881811270121, 0.8020957578842925])

In [ ]:

bbMax = np.array((1.5, 3, 1.5))
bbMin = np.array((-1.5, -1, -1.5))

dia2 = (bbMax - bbMin) * 0.5
center = bbMin + dia2

bsr = np.linalg.norm(dia2)

fov = pi/4
axis, angle = axon()
q = gp_Quaternion(axis, angle)

gp_viewDir = q.Multiply(gp_Vec(0,0,1))
viewDir = np.array((gp_viewDir.X(), gp_viewDir.Y(), gp_viewDir.Z()))
                        
tanfov2 = math.tan(fov / 2.0)
dist = bsr / tanfov2

position  = center + (viewDir * dist)
tanfov2, dist, center, bsr, viewDir, position

In [ ]:
def p(v):
    return (v.X(), v.Y(), v.Z())
    
# p(q.GetMatrix().Column(0)), p(q.GetMatrix().Column(1)), p(q.GetMatrix().Column(2))
p(q.Multiply(gp_Vec(0,0,1)))
center

In [ ]:


axis, angle = axon()
q = gp_Quaternion(axis, angle)

viewDir = q.Multiply(gp_Vec(0,0,1))

tanfov2 = math.tan(fov / 2.0)
dist = np.linalg.norm(dia2) / tanfov2

vec  = gp_Vec(0,1,0) + (viewDir * dist)

tanfov2, dist, p(gp_Vec(0,1,0)), p(viewDir), p(vec)